In [1]:
import os.path as op
import matplotlib.pyplot as plt

import numpy as np
from scipy.io import loadmat
from scipy.stats import ttest_rel, ttest_ind

from neurodsp.filt import filter_signal
from neurodsp.utils.norm import normalize_sig

from timescales.mp import compute_taus
from timescales.plts import plot_connected_scatter
from timescales.utils import create_windows

### Sliding Window Tau

#### Background
This notebook uses fcx-1, from CRCNS, to investigate sliding window tau estimation. This data consists of two cycles, each with ~7 minutes of wake followed by ~20 minutes of sleep. Based on Watson at al. 2016, they show that overall power increases, corresponding to low variance voltage segments, are associated with non-REM episodes. CRCNS provides times of sleep, non-rem, rem, and micro-arousals. Here, we focus on comparing timescales between sleep vs non-rem, since rem and micro arousals periods contain much less data.


#### Method / Results
For the ~54 minutes of interest, a window is slide across the data, with a step size of 1s. Each window contains 5s before and 5s after the current sample. Windows are grouped by wake and non-REM sleep periods. A PSD is computed for each window, and a timescale is determined. The results show that short timescales are associate with non-REM periods and long timescales are associated with wake periods.

#### Discussion
The wake periods tend to have short timescales, but also have lower r-squared values, compared to the non-REM periods. This may mean that there is more noise during the wake periods (i.e. noise from movement) and/or that the Lorenztian isn't the appriopriate model.

Many of the wake timescales appear to be > 1s long (< 1 Hz) and Welch's doesn't provide resolution below 1 Hz which may be something to also consider.

#### Reference
Network Homeostasis and State Dynamics of Neocortical Sleep. Watson BO, Levenstein D, Greene JP, Gelinas JN, Buzsáki G.; Neuron. 2016 Apr 27. pii: S0896-6273(16)30056-3. doi: 10.1016/j.neuron.2016.03.036

In [2]:
# Load LFP
data_dir = '/home/rph/Projects/timescale-methods/fcx1/data_mats'
lfp_file = op.join(data_dir, 'data21.mat')

fs = 1250
sig_lfp = loadmat(lfp_file)['data'][0]
sig_lfp = normalize_sig(sig_lfp, mean=0, variance=1)
sig_lfp = filter_signal(sig_lfp, fs, 'lowpass', (0, 200))
times = np.arange(0, len(sig_lfp)/fs, 1/fs)

# Extract behavioral data
beh_file = op.join(data_dir, '20140526_277um_WSRestrictedIntervals.mat')

beh = loadmat(beh_file)

nrem = beh['SWSPacketTimePairFormat'].astype(int) * fs
wake = beh['WakeTimePairFormat'].astype(int) * fs
rem = beh['REMTimePairFormat'].astype(int) * fs

# Settings
win_len = int(25*fs)
win_spacing = int(fs*25)

# Remove edges to reduce edge trial edge effects
wake_pad = wake.copy()
wake_pad[:, 0] = wake_pad[:, 0] + win_len
wake_pad[:, 1] = wake_pad[:, 1] - win_len

nrem_pad = nrem.copy()
nrem_pad[:, 0] = nrem_pad[:, 0] + win_len
nrem_pad[:, 1] = nrem_pad[:, 1] - win_len

# Create spectral windows
wake_starts, wake_mids, wake_ends = create_windows(wake_pad, win_len, win_spacing)
nrem_starts, nrem_mids, nrem_ends = create_windows(nrem_pad, win_len, win_spacing)

### SpecParam

In [3]:
#fooof_kwargs = dict(knee_bounds=(.05, 100000), max_n_peaks=8,
#                    peak_threshold=2, peak_width_limits=(1, 12))

fooof_kwargs = dict(knee_bounds=(.025, 10), peak_threshold=2.5, peak_width_limits=(4, 12))

fit_kwargs = dict(
    sig=sig_lfp,
    fs=fs,
    win_len=win_len,
    f_range=(1, 40),
    compute_spectrum_kwargs=dict(method='medfilt', avg_type='mean', filt_len=1.),
    #compute_spectrum_kwargs=dict(method='welch', avg_type='mean'),
    fit_kwargs=fooof_kwargs,
    rsq_type='linear'
)

wake_taus, wake_rsq, wake_fms = compute_taus(wake_starts, method='psd', fit_kwargs=fit_kwargs)
nrem_taus, nrem_rsq, nrem_fms = compute_taus(nrem_starts, method='psd', fit_kwargs=fit_kwargs)

  0%|          | 0/83 [00:00<?, ?it/s]

  0%|          | 0/74 [00:00<?, ?it/s]

### ACF

In [4]:
# ACF
bounds = [
    [1/(2*np.pi*10), None, None, None, None, None, None],
    [1/(2*np.pi*.025), None, None, None, None, None, None]
]

fit_kwargs = dict(
    sig = sig_lfp,
    fs = fs,
    win_len = win_len,
    method = 'cos',
    compute_acf_kwargs = dict(nlags=int(1 * fs)),
    fit_kwargs = dict(maxfev=10000, bounds=bounds)
)

wake_taus_acf, wake_rsq_acf, wake_acf_class = compute_taus(wake_starts, method='acf', fit_kwargs=fit_kwargs)
nrem_taus_acf, nrem_rsq_acf, nrem_acf_class = compute_taus(nrem_starts, method='acf', fit_kwargs=fit_kwargs)

  0%|          | 0/83 [00:00<?, ?it/s]

  0%|          | 0/74 [00:00<?, ?it/s]

### Results

In [ ]:
# Determine start and end of lfp
start_ends = [nrem[0][0], nrem[-1][-1], rem[0][0],
              rem[-1][-1], wake[0][0], wake[-1][-1]]

start = min(start_ends)-1
end = max(start_ends)

# Plot
fig, ax1 = plt.subplots(figsize=(12, 4))

ax1.set_xlabel('Time (s)')
ax1.set_ylabel('Normalized Voltage')
ax1.plot(times[start:end], sig_lfp[start:end], label='LFP', color='k', alpha=.5)
ax1.tick_params(axis='y', labelcolor='k')

mids = np.concatenate((wake_mids, nrem_mids)).astype(int)
taus_psd = np.concatenate((wake_taus, nrem_taus))
taus_acf = np.concatenate((wake_taus_acf, nrem_taus_acf))
order = np.argsort(mids)

ax2 = ax1.twinx()
ax2.set_ylabel('Tau')
ax2.plot(times[mids[order]], taus_psd[order], alpha=.8, color='C0', zorder=10, label='Tau PSD')
ax2.plot(times[mids[order]], taus_acf[order], alpha=.8, color='C1', zorder=10, label='Tau ACF')
ax2.tick_params(axis='y')

# Color regions
colors = ['C2', 'C3', 'C4']
labels = ['REM', 'Non-REM', 'Wake']

for periods, color, label in zip([rem, nrem, wake], colors, labels):
    for _ind, i in enumerate(periods):
        if _ind == 0:
            plt.axvspan(i[0]/fs, i[1]/fs, alpha=.25, color=color, zorder=3, label=label)
        else:
            plt.axvspan(i[0]/fs, i[1]/fs, alpha=.25, color=color, zorder=3)

plt.legend(loc='center left', bbox_to_anchor=(1.05, 0.5));

In [ ]:
# Remove rsq outliers
rsq_thresh = .5

wake_keep = np.where((wake_rsq >= rsq_thresh) & (wake_rsq_acf >= rsq_thresh))[0]
nrem_keep = np.where((nrem_rsq >= rsq_thresh) & (nrem_rsq_acf >= rsq_thresh))[0]

wake_fms = [i for ind, i in enumerate(wake_fms) if ind in wake_keep]
nrem_fms = [i for ind, i in enumerate(nrem_fms) if ind in nrem_keep]

wake_acf_class = [i for ind, i in enumerate(wake_acf_class) if ind in wake_keep]
nrem_acf_class = [i for ind, i in enumerate(nrem_acf_class) if ind in nrem_keep]

In [ ]:
fig, axes = plt.subplots(figsize=(15, 15), nrows=2, ncols=2)

# Each spectra/acf
for _fm in nrem_fms:
    axes[0][0].loglog(_fm.freqs, 10**_fm.power_spectrum, color='k', alpha=.05)

for _fm in wake_fms:
    axes[1][0].loglog(_fm.freqs, 10**_fm.power_spectrum, color='k', alpha=.05)

for i, _acf in enumerate(np.array([i.corrs for i in nrem_acf_class])):
    axes[0][1].plot(_acf, color='k', alpha=.05)

for i, _acf in enumerate(np.array([i.corrs for i in wake_acf_class])):
    axes[1][1].plot(_acf, color='k', alpha=.05)
    
# Mean spectra/acf
axes[0][0].plot(_fm.freqs, np.mean(np.array([10**i.power_spectrum for i in nrem_fms]), axis=0),
                color='C0')
axes[1][0].plot(_fm.freqs, np.mean(np.array([10**i.power_spectrum for i in wake_fms]), axis=0),
                color='C0')
axes[0][1].plot(np.mean(np.array([i.corrs for i in nrem_acf_class]), axis=0), color='C0')
axes[1][1].plot(np.mean(np.array([i.corrs for i in wake_acf_class]), axis=0), color='C0', label='Mean')
    
# Mean Fits
axes[0][0].plot(_fm.freqs, np.mean(np.array([10**i.fooofed_spectrum_ for i in nrem_fms]), axis=0),
                color='C1')
axes[1][0].plot(_fm.freqs, np.mean(np.array([10**i.fooofed_spectrum_ for i in wake_fms]), axis=0),
                color='C1')
axes[0][1].plot(np.mean(np.array([i.gen_corrs_fit() for i in nrem_acf_class]), axis=0), color='C1')
axes[1][1].plot(np.mean(np.array([i.gen_corrs_fit() for i in wake_acf_class]), axis=0), color='C1', label='Mean Fit')
    

axes[0][0].set_title('PSD', size=20)
axes[0][1].set_title('ACF', size=20)
axes[0][0].set_ylabel('Non-REM', size=20)
axes[1][0].set_ylabel('Wake', size=20)

plt.legend();

In [ ]:
# Run paired t-test
wake_rsq_pval = ttest_rel(wake_rsq[wake_keep], wake_rsq_acf[wake_keep]).pvalue
wake_taus_pval = ttest_rel(wake_taus[wake_keep], wake_taus_acf[wake_keep]).pvalue

nrem_rsq_pval = ttest_rel(nrem_rsq[nrem_keep], nrem_rsq_acf[nrem_keep]).pvalue
nrem_taus_pval = ttest_rel(nrem_taus[nrem_keep], nrem_taus_acf[nrem_keep]).pvalue

# Plot
fig, axes = plt.subplots(ncols=2, nrows=2, figsize=(15, 15))
plot_connected_scatter(wake_taus[wake_keep], wake_taus_acf[wake_keep], axes[0][0], 
                       f'Wake Tau: PSD vs ACF\nPaired T-Test P-Value = {wake_taus_pval}',
                       ylim=(0, 10))

plot_connected_scatter(nrem_taus[nrem_keep], nrem_taus_acf[nrem_keep], axes[0][1],
                       f'Non-REM Tau: PSD vs ACF\nPaired T-Test P-Value = {nrem_taus_pval}',
                       ylim=(0, 2))

plot_connected_scatter(wake_rsq[wake_keep], wake_rsq_acf[wake_keep], axes[1][0],
                       title=f'Wake R^2: PSD vs ACF\nPaired T-Test P-Value = {wake_rsq_pval}',
                       ylim=(.5, 1), ylabel='R^2')

plot_connected_scatter(nrem_rsq[nrem_keep], nrem_rsq_acf[nrem_keep], axes[1][1],
                       title=f'Non-REM R^2: PSD vs ACF\nPaired T-Test P-Value = {nrem_rsq_pval}',
                       ylim=(.5, 1), ylabel='R^2')

In [ ]:
# Run ind t-test
psd_taus_pval = ttest_ind(wake_taus, nrem_taus).pvalue
psd_rsq_pval = ttest_ind(wake_rsq, nrem_rsq).pvalue

acf_taus_pval = ttest_ind(wake_taus_acf[wake_keep], nrem_taus_acf[nrem_keep]).pvalue
acf_rsq_pval = ttest_ind(wake_rsq_acf[wake_keep], nrem_rsq_acf[nrem_keep]).pvalue

# Plot
fig, axes = plt.subplots(ncols=2, nrows=2, figsize=(15, 15))

plot_connected_scatter(wake_taus[wake_keep], nrem_taus[nrem_keep], axes[0][0], 
                       f'PSD Taus\nInd T-Test P-Value = {psd_taus_pval}',
                       ylim=(0, 5), ticklabels=['Wake', 'Non-REM'], paired=False)

plot_connected_scatter(wake_taus_acf[wake_keep], nrem_taus_acf[nrem_keep], axes[0][1],
                       f'ACF Taus\nInd T-Test P-Value = {acf_taus_pval}',
                       ylim=(0, 5), ticklabels=['Wake', 'Non-REM'], paired=False)

plot_connected_scatter(wake_rsq[wake_keep], nrem_rsq[nrem_keep], axes[1][0],
                       title=f'PSD R^2\nInd T-Test P-Value = {psd_rsq_pval}',
                       ylim=(.5, 1), ylabel='R^2', ticklabels=['Wake', 'Non-REM'], paired=False)

plot_connected_scatter(wake_rsq_acf[wake_keep], nrem_rsq_acf[nrem_keep], axes[1][1],
                       title=f'ACF R^2\nInd T-Test P-Value = {acf_rsq_pval}',
                       ylim=(.5, 1), ylabel='R^2', ticklabels=['Wake', 'Non-REM'], paired=False)